In [ ]:
'''code to read a set of logfiles and find information of runtime, memory, # of CPU/GPUs, etc.
 and write to a new document for the purpose of analysis of computing power and speed
 @author:wuaudrey'''

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install ase
import ase
from ase.io import read, write
import re

In [ ]:
'''GPU log files are slightly different, and this code will eventually be updated to account for that,
but currently this should only be used for CPU log files'''

machine_name = 'nalen'  # update these lines!
logfile_names = ['trial_sim.log',
                 'trial_sim_2.log'
                 ]


time = 'Total wall time:'
cpu = 'CPU use with'
memory = 'Per MPI rank memory allocation (min/avg/max) ='
neighbors = 'Ave neighs/atom'

def find_data(filename):
    # read all lines from the file
    with open(filename, "r") as f:
        lines = f.readlines()

    # find data
        for i, line in enumerate(lines):
            if time in line:
                wall_time = lines[i].strip().split("Total wall time:")[1].strip()
            if cpu in line:
              match = re.search(r'(\d+)\s+MPI tasks\s+x\s+(\d+)\s+OpenMP threads', line)
              if match:
                    mpi_tasks = match.group(1)
                    threads = match.group(2)

            '''most log files will have more than one mpi line, often one for the calculation of potentials
            and one for calculating the ion movement and effects. this code takes the mpi values for the last line
            written in the log file'''
            if memory in line:
              mpi = re.search(r'= (\d+\.\d+)\s*\|\s*(\d+\.\d+)\s*\|\s*(\d+\.\d+)', line)
              if mpi:
                    mpi_min = mpi.group(1)
                    mpi_avg = mpi.group(2)
                    mpi_max = mpi.group(3)
            if neighbors in line:
              avg_neigh = lines[i].strip().split('Ave neighs/atom =')[1].strip()

    return mpi_tasks, threads, wall_time, mpi_min, mpi_avg, mpi_max, avg_neigh

# write data to a new file
open("trial_sim_data.txt", 'x')
with open("trial_sim_data.txt", "w") as f:
    f.write(f"#machine name: {machine_name} \n")
    f.write(f"#{'Name'} \t {'CPUs'} \t {'Threads'} \t {'Wall Time'} \t {'MPI min'} \t {'MPI avg'} \t {'MPI max'} \t {'Avg Neighbors'}\n")
    for filename in logfile_names:
        mpi_tasks, threads, wall_time, mpi_min, mpi_avg, mpi_max, avg_neigh = find_data(filename)
        f.write(f"{filename} \t {mpi_tasks} \t {threads} \t {wall_time} \t {mpi_min} \t {mpi_avg} \t {mpi_max} \t {avg_neigh} \n")